# workflow_as_agent_human_in_the_loop.py - ELI5 Walkthrough
This notebook rebuilds `python/samples/getting_started/workflows/agents/workflow_as_agent_human_in_the_loop.py` in-place so you can explore it piece by piece.


## Big Picture
A workflow agent handles most reviews automatically. When the Reviewer executor lacks confidence, it emits a human review request. Your app receives a structured function call, collects the manager's decision, and feeds it back so the Worker can continue.


## Key Ingredients
- `WorkflowAgent` wraps a workflow so you can call it like any other chat agent.
- A `Worker` executor drafts answers, and a reviewer decides whether to escalate.

- `RequestInfoExecutor` pauses the workflow and surfaces a structured function call for the human.
- The driver code inspects messages for `WorkflowAgent.REQUEST_INFO_FUNCTION_NAME` and replies with a `FunctionResultContent`.


### Workflow Diagram
```mermaid
flowchart LR
    Start(["User Query"]) --> Worker[[Worker Executor]]
    Worker --> Reviewer[[Reviewer Executor]]
    Reviewer --> RequestInfo[/RequestInfoExecutor/]
    RequestInfo --> Human["Human Manager"]
    Human --> Reviewer
    Reviewer --> Worker
    Worker --> Output(["Agent Response Stream"])
```


### Step 1: Load dependencies and describe the scenario
We import the Agent Framework pieces, set up environment variables, and hold a short docstring summarising the workflow's goal.


In [ ]:
# Copyright (c) Microsoft. All rights reserved.
from dotenv import load_dotenv
load_dotenv()

import asyncio
from collections.abc import Mapping
from dataclasses import dataclass
from typing import Any, cast
from uuid import uuid4

from agent_framework import (
    AgentRunResponseUpdate,
    AgentRunUpdateEvent,
    ChatClientProtocol,
    ChatMessage,
    Contents,
    Executor,
    FunctionCallContent,
    FunctionResultContent,
    RequestInfoExecutor,
    RequestInfoMessage,
    RequestResponse,
    Role,
    WorkflowAgent,
    WorkflowBuilder,
    WorkflowContext,
    handler,
)
from agent_framework.openai import OpenAIChatClient
from pydantic import BaseModel

"""
Sample: Workflow Agent with Human-in-the-Loop

This walkthrough shows how to wrap a workflow as an agent and escalate tricky
cases to a human. A worker executor proposes answers, a reviewer escalates the
uncertain ones, and the workflow agent surfaces a structured function call so
your application can gather the human decision.
"""


### Step 2: Reuse the reflection pattern building blocks
We embed the `ReviewRequest`, `ReviewResponse`, and `Worker` classes from the reflection pattern sample. The Worker tracks pending requests, emits approved answers, and retries when feedback says "try again".


In [ ]:
@dataclass
class ReviewRequest:
    """Structured request passed from Worker to Reviewer for evaluation."""

    request_id: str
    user_messages: list[ChatMessage]
    agent_messages: list[ChatMessage]


@dataclass
class ReviewResponse:
    """Structured response from Reviewer back to Worker."""

    request_id: str
    feedback: str
    approved: bool


class Reviewer(Executor):
    """Executor that reviews agent responses and provides structured feedback."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client

    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext[ReviewResponse]) -> None:
        print(f"Reviewer: Evaluating response for request {request.request_id[:8]}...")

        # Define structured schema for the LLM to return.
        class _Response(BaseModel):
            feedback: str
            approved: bool

        # Construct review instructions and context.
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text=(
                    "You are a reviewer for an AI agent. Provide feedback on the "
                    "exchange between a user and the agent. Indicate approval only if:\n"
                    "- Relevance: response addresses the query\n"
                    "- Accuracy: information is correct\n"
                    "- Clarity: response is easy to understand\n"
                    "- Completeness: response covers all aspects\n"
                    "Do not approve until all criteria are satisfied."
                ),
            )
        ]
        # Add conversation history.
        messages.extend(request.user_messages)
        messages.extend(request.agent_messages)

        # Add explicit review instruction.
        messages.append(ChatMessage(role=Role.USER, text="Please review the agent's responses."))

        print("Reviewer: Sending review request to LLM...")
        response = await self._chat_client.get_response(messages=messages, response_format=_Response)

        parsed = _Response.model_validate_json(response.messages[-1].text)

        print(f"Reviewer: Review complete - Approved: {parsed.approved}")
        print(f"Reviewer: Feedback: {parsed.feedback}")

        # Send structured review result to Worker.
        await ctx.send_message(
            ReviewResponse(request_id=request.request_id, feedback=parsed.feedback, approved=parsed.approved)
        )


class Worker(Executor):
    """Executor that generates responses and incorporates feedback when necessary."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client
        self._pending_requests: dict[str, tuple[ReviewRequest, list[ChatMessage]]] = {}

    @handler
    async def handle_user_messages(self, user_messages: list[ChatMessage], ctx: WorkflowContext[ReviewRequest]) -> None:
        print("Worker: Received user messages, generating response...")

        # Initialize chat with system prompt.
        messages = [ChatMessage(role=Role.SYSTEM, text="You are a helpful assistant.")]
        messages.extend(user_messages)

        print("Worker: Calling LLM to generate response...")
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: Response generated: {response.messages[-1].text}")

        # Add agent messages to context.
        messages.extend(response.messages)

        # Create review request and send to Reviewer.
        request = ReviewRequest(request_id=str(uuid4()), user_messages=user_messages, agent_messages=response.messages)
        print(f"Worker: Sending response for review (ID: {request.request_id[:8]})")
        await ctx.send_message(request)

        # Track request for possible retry.
        self._pending_requests[request.request_id] = (request, messages)

    @handler
    async def handle_review_response(self, review: ReviewResponse, ctx: WorkflowContext[ReviewRequest]) -> None:
        print(f"Worker: Received review for request {review.request_id[:8]} - Approved: {review.approved}")

        if review.request_id not in self._pending_requests:
            raise ValueError(f"Unknown request ID in review: {review.request_id}")

        request, messages = self._pending_requests.pop(review.request_id)

        if review.approved:
            print("Worker: Response approved. Emitting to external consumer...")
            contents: list[Contents] = []
            for message in request.agent_messages:
                contents.extend(message.contents)

            # Emit approved result to external consumer via AgentRunUpdateEvent.
            await ctx.add_event(
                AgentRunUpdateEvent(self.id, data=AgentRunResponseUpdate(contents=contents, role=Role.ASSISTANT))
            )
            return

        print(f"Worker: Response not approved. Feedback: {review.feedback}")
        print("Worker: Regenerating response with feedback...")

        # Incorporate review feedback.
        messages.append(ChatMessage(role=Role.SYSTEM, text=review.feedback))
        messages.append(
            ChatMessage(role=Role.SYSTEM, text="Please incorporate the feedback and regenerate the response.")
        )
        messages.extend(request.user_messages)

        # Retry with updated prompt.
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: New response generated: {response.messages[-1].text}")

        messages.extend(response.messages)

        # Send updated request for re-review.
        new_request = ReviewRequest(
            request_id=review.request_id, user_messages=request.user_messages, agent_messages=response.messages
        )
        await ctx.send_message(new_request)

        # Track new request for further evaluation.
        self._pending_requests[new_request.request_id] = (new_request, messages)




### Step 3: Escalate reviews to a human manager
`ReviewerWithHumanInTheLoop` sends every review to a `RequestInfoExecutor`, waits for the human's `ReviewResponse`, and forwards it back to the Worker.


In [ ]:
@dataclass
class HumanReviewRequest(RequestInfoMessage):
    """Payload escalated to a human reviewer."""

    agent_request: ReviewRequest | None = None


class ReviewerWithHumanInTheLoop(Executor):
    """Reviewer that always escalates to a human manager."""

    def __init__(self, worker_id: str, request_info_id: str, reviewer_id: str | None = None) -> None:
        unique_id = reviewer_id or f"{worker_id}-reviewer"
        super().__init__(id=unique_id)
        self._worker_id = worker_id
        self._request_info_id = request_info_id

    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext[ReviewResponse | HumanReviewRequest]) -> None:
        print(f"Reviewer: Evaluating response for request {request.request_id[:8]}...")
        print("Reviewer: Escalating to human manager...")
        await ctx.send_message(HumanReviewRequest(agent_request=request), target_id=self._request_info_id)

    @handler
    async def accept_human_review(
        self,
        response: RequestResponse[HumanReviewRequest, ReviewResponse],
        ctx: WorkflowContext[ReviewResponse],
    ) -> None:
        human_response = response.data
        assert isinstance(human_response, ReviewResponse)
        print(f"Reviewer: Accepting human review for request {human_response.request_id[:8]}...")
        print(f"Reviewer: Human feedback: {human_response.feedback}")
        print(f"Reviewer: Human approved: {human_response.approved}")
        print("Reviewer: Forwarding human review back to worker...")
        await ctx.send_message(human_response, target_id=self._worker_id)


### Step 4: Wrap the workflow as an agent and drive the human loop
The `main()` coroutine builds the workflow agent, sends an initial query, detects when human input is required, and feeds back a mocked approval. Replace the mock with real UI input in your app.


In [ ]:
async def main() -> None:
    print("Starting Workflow Agent with Human-in-the-Loop Demo")
    print("=" * 50)

    # Create executors for the workflow.
    print("Creating chat client and executors...")
    mini_chat_client = OpenAIChatClient(model_id="gpt-4.1-nano")
    worker = Worker(id="sub-worker", chat_client=mini_chat_client)
    request_info_executor = RequestInfoExecutor(id="request_info")
    reviewer = ReviewerWithHumanInTheLoop(worker_id=worker.id, request_info_id=request_info_executor.id)

    print("Building workflow with Worker ↔ Reviewer cycle...")
    # Build a workflow with bidirectional communication between Worker and Reviewer,
    # and escalation paths for human review.
    agent = (
        WorkflowBuilder()
        .add_edge(worker, reviewer)  # Worker sends requests to Reviewer
        .add_edge(reviewer, worker)  # Reviewer sends feedback to Worker
        .add_edge(reviewer, request_info_executor)  # Reviewer requests human input
        .add_edge(request_info_executor, reviewer)  # Human input forwarded back to Reviewer
        .set_start_executor(worker)
        .build()
        .as_agent()  # Convert workflow into an agent interface
    )

    print("Running workflow agent with user query...")
    print("Query: 'Write code for parallel reading 1 million files on disk and write to a sorted output file.'")
    print("-" * 50)

    # Run the agent with an initial query.
    response = await agent.run(
        "Write code for parallel reading 1 million Files on disk and write to a sorted output file."
    )

    # Locate the human review function call in the response messages.
    human_review_function_call: FunctionCallContent | None = None
    for message in response.messages:
        for content in message.contents:
            if isinstance(content, FunctionCallContent) and content.name == WorkflowAgent.REQUEST_INFO_FUNCTION_NAME:
                human_review_function_call = content

    # Handle the human review if required.
    if human_review_function_call:
        # Parse the human review request arguments.
        human_request_args = human_review_function_call.arguments
        if isinstance(human_request_args, str):
            request: WorkflowAgent.RequestInfoFunctionArgs = WorkflowAgent.RequestInfoFunctionArgs.from_json(
                human_request_args
            )
        elif isinstance(human_request_args, Mapping):
            request = WorkflowAgent.RequestInfoFunctionArgs.from_dict(dict(human_request_args))
        else:
            raise TypeError("Unexpected argument type for human review function call.")

        request_payload_obj: Any = request.data
        if not isinstance(request_payload_obj, Mapping):
            raise ValueError("Human review request payload must be a mapping.")
        request_payload = cast(Mapping[str, Any], request_payload_obj)

        agent_request_obj = request_payload.get("agent_request")
        if not isinstance(agent_request_obj, Mapping):
            raise ValueError("Human review request must include agent_request mapping data.")
        agent_request_data = cast(Mapping[str, Any], agent_request_obj)

        request_id_obj = agent_request_data.get("request_id")
        if not isinstance(request_id_obj, str):
            raise ValueError("Human review request_id must be a string.")
        request_id_value = request_id_obj

        # Mock a human response approval for demonstration purposes.
        human_response = ReviewResponse(request_id=request_id_value, feedback="Approved", approved=True)

        # Create the function call result object to send back to the agent.
        human_review_function_result = FunctionResultContent(
            call_id=human_review_function_call.call_id,
            result=human_response,
        )
        # Send the human review result back to the agent.
        response = await agent.run(ChatMessage(role=Role.TOOL, contents=[human_review_function_result]))
        print(f"📤 Agent Response: {response.messages[-1].text}")

    print("=" * 50)
    print("Workflow completed!")




### Step ??: Try it yourself
Use the helper below. In notebooks it awaits `main()` on the active loop; in scripts it falls back to `asyncio.run(main())`.


In [ ]:
import asyncio

# Helper for notebooks vs. scripts
loop = asyncio.get_event_loop()
if loop.is_running():
    # Jupyter/VS Code notebooks already have an event loop, so await directly.
    await main()
else:
    asyncio.run(main())
